<a href="https://colab.research.google.com/github/Madhu-712/Chip_designer_copilot_Agent/blob/main/LangGraph/Chip_design_LangGraph_Multi_Agent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MultiAgent LangGraph Agent
In this notebook multiple Agents pattern is illustrated .Here different  agents work with one another to  accomplish complex tasks execution.

In [ ]:
pip install -U langgraph langchain-experimental langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain_groq langchain groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.2 MB/s eta 0:00:00


##Importing API keys

In [ ]:
import getpass
import os
os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ_API_KEY:\n")

GROQ_API_KEY:
··········


In [ ]:
gsk_9Hvs8nkYUZ7sf5Q1KQbnWGdyb3FYEsw2GFgvM638fKuIxBa4uKyhuyul

In [ ]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

TAVILY_API_KEY:
··········


In [ ]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str

#Define Agents

In [ ]:
from typing import Literal
from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage
#from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState, END
from langgraph.types import Command

llm = ChatGroq(model="Llama3-8b-8192",temperature=0.0)
#llm = ChatAnthropic(model="claude-3-5-sonnet-latest")
#llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)
def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto
def orchestrator_agent(role, instructions):
    prompt = ChatPromptTemplate.from_messages(
        [("system", f"You are a {role}. {instructions}"), ("human", "{input}")]
    )
    # Assuming llm is defined in the scope where this function is called
    return prompt | llm
def orchestrator_node(
    state: MessagesState,
) -> Command[Literal["Orchestrator", END]]:
    result = orchestrator_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "Orchestrator")
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="Orchestrator"
    )
    return Command(
        update={
            # share internal message history of orchestrator agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )

In [ ]:
def AI_model_agent(role,instructions):
    prompt=ChatPromptTemplate.from_messages(
         [("system", f"You are a {role}. {instructions}"), ("human", "{input}")]
    )
    return prompt | llm
def AI_model_node(
    state: MessagesState,
) -> Command[Literal["AI model", END]]:
    result = AI_model_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "AI model")
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="AI model"
    )
    return Command(
        update={
            # share internal message history of AI model agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )

In [ ]:
def Design_agent(role,instructions):
    prompt=ChatPromptTemplate.from_messages(
         [("system", f"You are a {role}. {instructions}"), ("human", "{input}")]
    )
    return prompt | llm
def Design_node(
    state: MessagesState,
) -> Command[Literal["Design", END]]:
    result = Design_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "Design")
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="Design"
    )
    return Command(
        update={
            # share internal message history of Design agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )

In [ ]:
def create_agent(role, instructions):
    prompt = ChatPromptTemplate.from_messages(
        [("system", f"You are a {role}. {instructions}"), ("human", "{input}")]
    )
    # Assuming llm is defined in the scope where this function is called
    return prompt | llm

In [ ]:
Orchestrator= create_agent("Orchestrator", "Orchestratea AI agent and design agent and prepares final chip design report")
AI_model = create_agent("AI model", "Analyze the chip architecture  and suggest power,area optimization and improvements")
Design = create_agent("Design", "Review & Implement the suggested improvements in the chip design and generate a formal chip design report")

In [ ]:
from typing import Dict
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END  # Import START

##Create and compile the workflow

In [ ]:
workflow = StateGraph(state_schema=dict)  # Initialize without 'nodes'
# Add nodes individually using add_node
workflow.add_node("Orchestrator", Orchestrator)
workflow.add_node("AI model", AI_model) #Corrected this line
workflow.add_node("Design", Design)


# Define the flow of our multi-agent system, potentially starting from START
workflow.add_edge(START, "Orchestrator")  # If 'coder' is the initial node
workflow.add_edge("Orchestrator", "AI model")
workflow.add_edge("AI model", "Design")
workflow.add_edge("Design", END)

# Compile the workflow
chain = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(chain.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
task = "how to choose the no. Of input  pins in a Chip architecture and Generate a report emphasizing this?"

result = chain.invoke({"input": task})
print(result)

content='**Formal Chip Design Report**\n\n**Introduction:**\n\nThis report presents the design improvements and optimizations suggested for the provided chip architecture. The report is divided into three main sections: Power Optimization, Area Optimization, and Improvements. The suggested improvements aim to reduce power consumption, minimize area, and enhance overall chip performance.\n\n**Power Optimization:**\n\n1. **Clock Gating:** Implement clock gating to reduce power consumption by disabling clock signals to unused logic blocks. This technique can reduce power consumption by up to 30%.\n2. **Power Domains:** Divide the chip into power domains, allowing for independent power management and reducing power consumption. This technique can reduce power consumption by up to 20%.\n3. **Voltage Scaling:** Implement voltage scaling to reduce power consumption during idle periods or when the chip is not performing critical tasks. This technique can reduce power consumption by up to 15%.\

In [ ]:
from IPython.display import Markdown

# Access the content of the AIMessage object, which is a string
markdown_content = result.content

# Now pass the string content to Markdown
print(Markdown(markdown_content))

<IPython.core.display.Markdown object>


In [ ]:
from IPython.display import display, Markdown
display(Markdown(str(result)))

content='**Formal Chip Design Report**\n\n**Introduction:**\n\nThis report presents the design improvements and optimizations suggested for the provided chip architecture. The report is divided into three main sections: Power Optimization, Area Optimization, and Improvements. The suggested improvements aim to reduce power consumption, minimize area, and enhance overall chip performance.\n\n**Power Optimization:**\n\n1. **Clock Gating:** Implement clock gating to reduce power consumption by disabling clock signals to unused logic blocks. This technique can reduce power consumption by up to 30%.\n2. **Power Domains:** Divide the chip into power domains, allowing for independent power management and reducing power consumption. This technique can reduce power consumption by up to 20%.\n3. **Voltage Scaling:** Implement voltage scaling to reduce power consumption during idle periods or when the chip is not performing critical tasks. This technique can reduce power consumption by up to 15%.\n4. **Leakage Reduction:** Implement techniques to reduce leakage current, such as transistor sizing and body biasing. This technique can reduce power consumption by up to 10%.\n\n**Area Optimization:**\n\n1. **Logic Minimization:** Optimize the logic design to minimize the number of gates and reduce area. This technique can reduce area by up to 20%.\n2. **Register Allocation:** Optimize register allocation to reduce the number of registers and minimize area. This technique can reduce area by up to 15%.\n3. **Memory Optimization:** Optimize memory usage by reducing the number of memory accesses and using more efficient memory structures. This technique can reduce area by up to 10%.\n4. **Floorplanning:** Optimize floorplanning to reduce wirelength and minimize area. This technique can reduce area by up to 5%.\n\n**Improvements:**\n\n1. **Data Path Optimization:** Optimize the data path to reduce the number of stages and minimize area. This technique can reduce area by up to 10%.\n2. **Control Path Optimization:** Optimize the control path to reduce the number of control signals and minimize area. This technique can reduce area by up to 5%.\n3. **Clock Distribution Network:** Optimize the clock distribution network to reduce skew and minimize area. This technique can reduce area by up to 5%.\n4. **Power Distribution Network:** Optimize the power distribution network to reduce voltage drops and minimize area. This technique can reduce area by up to 5%.\n\n**Additional Recommendations:**\n\n1. **Use a more efficient instruction set architecture (ISA):** Consider using a more efficient ISA to reduce the number of instructions and minimize area.\n2. **Use a more efficient memory hierarchy:** Consider using a more efficient memory hierarchy, such as a cache hierarchy, to reduce memory accesses and minimize area.\n3. **Use a more efficient clocking scheme:** Consider using a more efficient clocking scheme, such as a dual-edge-triggered clock, to reduce clock skew and minimize area.\n4. **Use a more efficient power management scheme:** Consider using a more efficient power management scheme, such as dynamic voltage and frequency scaling, to reduce power consumption and minimize area.\n\n**Conclusion:**\n\nThe suggested improvements and optimizations aim to reduce power consumption, minimize area, and enhance overall chip performance. By implementing these suggestions, the chip architecture can be optimized for better performance, power efficiency, and area utilization.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 670, 'prompt_tokens': 721, 'total_tokens': 1391, 'completion_time': 0.558333333, 'prompt_time': 0.099409284, 'queue_time': 0.018439843999999997, 'total_time': 0.657742617}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-0fa38319-bd02-44db-9010-64dd4e03ba78-0' usage_metadata={'input_tokens': 721, 'output_tokens': 670, 'total_tokens': 1391}